In [0]:
# Import required modules
import cv2 as cv
import math
import argparse
import pytesseract

############ Add argument parser for command line arguments ############
parser = argparse.ArgumentParser(description='Use this script to run TensorFlow implementation (https://github.com/argman/EAST) of EAST: An Efficient and Accurate Scene Text Detector (https://arxiv.org/abs/1704.03155v2)')
parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')
parser.add_argument('--model', required=True,
                    help='Path to a binary .pb file of model contains trained weights.')
parser.add_argument('--width', type=int, default=320,
                    help='Preprocess input image by resizing to a specific width. It should be multiple by 32.')
parser.add_argument('--height', type=int, default=320,
                    help='Preprocess input image by resizing to a specific height. It should be multiple by 32.')
parser.add_argument('--thr', type=float, default=0.5,
                    help='Confidence threshold.')
parser.add_argument('--nms', type=float, default=0.4,
                    help='Non-maximum suppression threshold.')
args = parser.parse_args()

############ Utility functions ############
def decode(scores, geometry, scoreThresh):
    detections = []
    confidences = []

    ############ CHECK DIMENSIONS AND SHAPES OF geometry AND scores ############
    assert len(scores.shape) == 4, "Incorrect dimensions of scores"
    assert len(geometry.shape) == 4, "Incorrect dimensions of geometry"
    assert scores.shape[0] == 1, "Invalid dimensions of scores"
    assert geometry.shape[0] == 1, "Invalid dimensions of geometry"
    assert scores.shape[1] == 1, "Invalid dimensions of scores"
    assert geometry.shape[1] == 5, "Invalid dimensions of geometry"
    assert scores.shape[2] == geometry.shape[2], "Invalid dimensions of scores and geometry"
    assert scores.shape[3] == geometry.shape[3], "Invalid dimensions of scores and geometry"
    height = scores.shape[2]
    width = scores.shape[3]
    for y in range(0, height):

        # Extract data from scores
        scoresData = scores[0][0][y]
        x0_data = geometry[0][0][y]
        x1_data = geometry[0][1][y]
        x2_data = geometry[0][2][y]
        x3_data = geometry[0][3][y]
        anglesData = geometry[0][4][y]
        for x in range(0, width):
            score = scoresData[x]

            # If score is lower than threshold score, move to next x
            if(score < scoreThresh):
                continue

            # Calculate offset
            offsetX = x * 4.0
            offsetY = y * 4.0
            angle = anglesData[x]

            # Calculate cos and sin of angle
            cosA = math.cos(angle)
            sinA = math.sin(angle)
            h = x0_data[x] + x2_data[x]
            w = x1_data[x] + x3_data[x]

            # Calculate offset
            offset = ([offsetX + cosA * x1_data[x] + sinA * x2_data[x], offsetY - sinA * x1_data[x] + cosA * x2_data[x]])

            # Find points for rectangle
            p1 = (-sinA * h + offset[0], -cosA * h + offset[1])
            p3 = (-cosA * w + offset[0],  sinA * w + offset[1])
            center = (0.5*(p1[0]+p3[0]), 0.5*(p1[1]+p3[1]))
            detections.append((center, (w,h), -1*angle * 180.0 / math.pi))
            confidences.append(float(score))

    # Return detections and confidences
    return [detections, confidences]

def main():
    # Read and store arguments
    confThreshold = args.thr
    nmsThreshold = args.nms
    inpWidth = args.width
    inpHeight = args.height
    model = args.model

    # Load network
    net = cv.dnn.readNet(model)

    # Create a new named window
    kWinName = "EAST: An Efficient and Accurate S/cene Text Detector"
    # cv.namedWindow(kWinName, cv.WINDOW_NORMAL)
    outNames = []
    outNames.append("feature_fusion/Conv_7/Sigmoid")
    outNames.append("feature_fusion/concat_3")

    # Open a video file or an image file or a camera stream
    cap = cv.VideoCapture(args.input if args.input else 0)

    while cv.waitKey(1) < 0:
        # Read frame
        hasFrame, frame = cap.read()
        if not hasFrame:
            cv.waitKey()
            break

        # Get frame height and width
        height_ = frame.shape[0]
        width_ = frame.shape[1]
        rW = width_ / float(inpWidth)
        rH = height_ / float(inpHeight)

        # Create a 4D blob from frame.
        image1 = frame.copy()
        blob = cv.dnn.blobFromImage(frame, 1.0, (inpWidth, inpHeight), (123.68, 116.78, 103.94), True, False)

        # Run the model
        net.setInput(blob)
        outs = net.forward(outNames)
        t, _ = net.getPerfProfile()
        label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())

        # Get scores and geometry
        scores = outs[0]
        geometry = outs[1]
        [boxes, confidences] = decode(scores, geometry, confThreshold)

        # Apply NMS
        indices = cv.dnn.NMSBoxesRotated(boxes, confidences, confThreshold,nmsThreshold)
        # print("indices  nms: ", indices)
        # print("boxes nms: ", boxes)
        for i in indices:
            # get 4 corners of the rotated rect
            val = boxes[i[0]]
            # print(" val: ", val)

            rotation_matrix = cv.getRotationMatrix2D((val[0][0]*rW, val[0][1]*rH), val[2], 1)
            rotated_image = cv.warpAffine(image1, rotation_matrix, (width_, height_))
            x=int(val[0][0]*rW)
            y=int(val[0][1]*rH)
            w=int(val[1][0]*rW)
            h=int(val[1][1]*rH)

            l1 = (int(x - (w / 2)), int(y - (h / 2)))
            l2 = (int(x + (w / 2)), int(y - (h / 2)))
            l3 = (int(x + (w / 2)), int(y + (h / 2)))
            l4 = (int(x - (w / 2)), int(y + (h / 2)))

            # cv.line(rotated_image, l1, l2, (0, 255, 0), 1)
            # cv.line(rotated_image, l2, l3, (0, 255, 0), 1)
            # cv.line(rotated_image, l3, l4, (0, 255, 0), 1)
            # cv.line(rotated_image, l4, l1, (0, 255, 0), 1)

            stry=l1[1]
            endy=l4[1]
            strtx=l1[0]
            endx=l2[0]

            weigh = 0.0
            wid = weigh * rW
            ht = weigh * rH
            dX = int((endx - strtx) * weigh)
            dY = int((endy - stry) * weigh)

            strtx = max(0, strtx - dX)
            stry = max(0, stry - dY)
            endx = min(width_, endx + (dX * 2))
            endy = min(height_, endy + (dY * 2))

            newimg= rotated_image[stry:endy, strtx:endx]
            # print(" my vertices: ", stry, endy, strtx,  endx)
            cv.imshow("rotated_image", newimg)
            cv.waitKey(0)

            row, col = newimg.shape[:2]
            bottom = newimg[row - 2:row, 0:col]
            mean = cv.mean(bottom)[0]

            bordersize = 10
            border = cv.copyMakeBorder(newimg, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize,
                                       borderType=cv.BORDER_CONSTANT, value=[mean, mean, mean])

            config = ("-l eng --oem 1 --psm 7")
            text = pytesseract.image_to_string(border, config=config)
            # cv.putText(rotated_image, text, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
            cv.imshow("border_rotated_image", border)
            cv.waitKey(0)
            print(" text is:", text)
            vertices = cv.boxPoints(boxes[i[0]])
            # print("vertices: ", vertices)
            # scale the bounding box coordinates based on the respective ratios
        #     for j in range(4):
        #         vertices[j][0] *= rW
        #         vertices[j][1] *= rH
        #     for j in range(4):
        #         p1 = (vertices[j][0], vertices[j][1])
        #         p2 = (vertices[(j + 1) % 4][0], vertices[(j + 1) % 4][1])
        #         cv.line(frame, p1, p2, (0, 255, 0), 1);
        #
        # # Put efficiency information
        # cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

        # Display the frame
        # cv.imshow(kWinName,frame)

if __name__ == "__main__":
    main()